In [ ]:
#!/usr/bin/env python3

import re
import sys

import numpy as np
import matplotlib.pyplot as plt

from skimage import io
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb, rgb2gray
from skimage.filters import try_all_threshold, threshold_isodata, threshold_minimum, threshold_otsu

def FluCount (Filename = '14251.5_3(2).jpg', Bright = False):
    #This function xxx
    
    image = io.imread(Filename)
    
    green_image = image.copy()
    
    # Change to grayscale according to green channel
    green_image[:,:,0] = 0
    green_image[:,:,2] = 0
    gray_image = rgb2gray(green_image)
    
    # Set threshold to get binary images (for all fluorescent seeds and bright seeds)
    if Bright == False:
        thresh = threshold_otsu(gray_image)
    else:
        thresh = threshold_minimum(gray_image)
    binary = gray_image > thresh
    
    # Label regions
    bw = closing(binary, square(3)) 
    cleared = clear_border(bw) #ignore all the seeds at the edge of the image
    label_image = label(cleared)
    seed = 0
    cali_seed = 0
    cali_area = 0
    large_area = 0

    for region in regionprops(label_image):
        if 400 < region.area < 700 :
            cali_seed = cali_seed + 1
            cali_area = cali_area + region.area
        if 5 < region.area < 700:
            #print('label ' + str(region.label) + ' area ' + str(region.area))
            seed = seed + 1
        elif region.area > 700:
            large_area = large_area + region.area

    ave_area = cali_area / cali_seed
    large_seed = large_area / ave_area
    seed = seed + large_seed

    #plt.figure(figsize=(20,10))
    #plt.imshow(image)
    #plt.imshow(binary)

    print('average area of seeds is: ' + str(ave_area))
    print('There are ' + str(int(large_seed)) + ' large seeds')
    print('Total seeds: ' + str(int(seed)))

